# 리디북스 일반도서 베스트 셀러
## 작업순서 잘알기!
 - https://ridibooks.com/bestsellers/general?order=monthly&page=1  
 - https://ridibooks.com/robots.txt   <-  확인해보니 제목가져오는건 아무 상관이 없구나!  
 - 페이지 처리는 get방식이구나~ for문돌려서 페이지넘기면 되겠구나!
 - 책위에 별표시 붙어있는책이름 원씽 밑에도 있어. 중복데이터를 가져오면 다꼬이기 때문에 저거 처리를 해야겠구나!  
 - selector로 안되면 바로 tag로 넘어가야함

In [1]:
# improt를 쓸때 어디에 뭐쓰는지 알아야 나중에 빅데이터 다룰때 퍼포먼스 영향이 커진다.
# 너무 다 불러오면 느려져
from bs4 import BeautifulSoup
import urllib.request as req

url = "https://ridibooks.com/bestsellers/general?order=monthly&page=1"
res = req.urlopen(url)
soup = BeautifulSoup(res, "html.parser")

#내가 가져온게 자바스크립트인지 아닌지를 확인하는 단계!(자바 스크립트 실행전단계에서 확인)========================
#내가 가져온게 있으면 자바스크립트 쓴게 아님(찾는게 없으면 자바스크립트)
# print(soup)

soup.select("#page_best > div.book_macro_wrapper.js_book_macro_wrapper > div.book_macro_110.book_macro_portrait.book_macro_column_6.recommended_book > div.book_metadata_wrapper > h3 > a > span")

[<span class="title_text js_highlight_helper hidden">          
                         원씽
             </span>]

> copy seletor로 가져오니 class 이름이 보임

# selector를 간단하게 사용하기!

In [27]:
# improt를 쓸때 어디에 뭐쓰는지 알아야 나중에 빅데이터 다룰때 퍼포먼스 영향이 커진다.
# 너무 다 불러오면 느려져
from bs4 import BeautifulSoup
import urllib.request as req

url = "https://ridibooks.com/bestsellers/general?order=monthly&page=1"
res = req.urlopen(url)
soup = BeautifulSoup(res, "html.parser")

#내가 가져온게 자바스크립트인지 아닌지를 확인하는 단계!(자바 스크립트 실행전단계에서 확인)========================
#내가 가져온게 있으면 자바스크립트 쓴게 아님(찾는게 없으면 자바스크립트)
# print(soup)

scraping_lists = soup.select(".title_text")
#print(scraping_lists)

### 책권수 확인하기

In [28]:
len(scraping_lists)

31

> 별표 붙은 책때문에 31권을 가져왔음

### 첫번째  책을 제거후 List를 재구성하기

In [29]:
scraping_lists = scraping_lists[1:]
len(scraping_lists)
print(scraping_lists[0])

<span class="title_text js_highlight_helper hidden">          
                        역행자
            </span>


In [5]:
book_lists = []
for book_list in scraping_lists:
    book_lists.append(book_list.string.strip())
    
print(book_lists)

['역행자', '작별인사', '인플레이션에서 살아남기', '변화하는 세계 질서', '피스타운', '악마의 계약서는 만기 되지 않는다', '내가 틀릴 수도 있습니다', '불편한 편의점', '개정판 | 하얀 늑대들', '최재천의 공부', '잠시 동안 빛은 이곳에', '친밀한 이방인', '사이후이, 싸이파이', '심해어', '물고기는 존재하지 않는다', '도파민네이션', '멘탈을 바꿔야 인생이 바뀐다', '구글 엔지니어는 이렇게 일한다', '플라스틱 세대', '덫', '나의 투자는 새벽 4시에 시작된다', '레스토랑 안의 낯선 자들', '원씽', '버드나무', '브라질에 비가 내리면 스타벅스 주식을 사라', '모든 것은 기본에서 시작한다', '부자 아빠 가난한 아빠 20주년 특별 기념판', '밝은 밤', '나를 단단하게 만드는 심리학', '특별한 아이, 특별한 엄마']


# DataFrame 만들기

In [6]:
# Rank List 만들기
rank = list(range(1,31))

## 처음부터 여러데이터 넣어서 만들기

In [31]:
data = []

for i,j in zip(rank, book_lists):
    data.append([i, j])

In [8]:
# 데이터가 2차원 배열로 만들어져있음. (컬럼이 두개라)
data[:5]

[[1, '역행자'], [2, '작별인사'], [3, '인플레이션에서 살아남기'], [4, '변화하는 세계 질서'], [5, '피스타운']]

In [9]:
# 데이터 프레임 만들때 필요
import pandas as pd
page1 = pd.DataFrame(data=data, columns=[['Rank', 'bookTitle']])
# 데이터 프레임이니까 헤드 쓸수 있다.
page1.head()

,Rank,bookTitle
0,1,역행자
1,2,작별인사
2,3,인플레이션에서 살아남기
3,4,변화하는 세계 질서
4,5,피스타운


In [10]:
# csv 로 저장
page1.to_csv("../Data/ridi_page1.csv", index=False)
ridibooks = pd.read_csv("../Data/ridi_page1.csv").head()

---
# 작품 소개 가져오기

## 1page의 도서명과 hyperlink를 통한 작품 소개 내용 가져오기

### Hyper link 정보 가져오기

In [34]:
# improt를 쓸때 어디에 뭐쓰는지 알아야 나중에 빅데이터 다룰때 퍼포먼스 영향이 커진다.
# 너무 다 불러오면 느려져
from bs4 import BeautifulSoup
import urllib.request as req

url = "https://ridibooks.com/bestsellers/general?order=monthly&page=1"
res = req.urlopen(url)
soup = BeautifulSoup(res, "html.parser")

##page_best > div.book_macro_wrapper.js_book_macro_wrapper > div:nth-child(3) > div.book_metadata_wrapper > h3 > a
hf_list = soup.select(".title_link")
hf_list = hf_list[1:]
print(hf_list[1:5])

[<a class="title_link" data-track-params="" data-track-type="" href="/books/4271000014?_rdt_sid=bestseller&amp;_rdt_idx=1">
<span class="title_text js_highlight_helper hidden">          
                        작별인사
            </span>
</a>, <a class="title_link" data-track-params="" data-track-type="" href="/books/4139000033?_rdt_sid=bestseller&amp;_rdt_idx=2">
<span class="title_text js_highlight_helper hidden">          
                        인플레이션에서 살아남기
            </span>
</a>, <a class="title_link" data-track-params="" data-track-type="" href="/books/1890000285?_rdt_sid=bestseller&amp;_rdt_idx=3">
<span class="title_text js_highlight_helper hidden">          
                        변화하는 세계 질서
            </span>
</a>, <a class="title_link" data-track-params="" data-track-type="" href="/books/4575000003?_rdt_sid=bestseller&amp;_rdt_idx=4">
<span class="title_text js_highlight_helper hidden">          
                        피스타운
            </span>
</a>]


In [12]:
hyper_lists = []
for span in hf_list:
    hyper_lists = span.attrs['href']
    #hyper_lists.append(hyper_list.string.strip())
    print(hyper_lists)
    

/books/606002388?_rdt_sid=bestseller&_rdt_idx=0
/books/4271000014?_rdt_sid=bestseller&_rdt_idx=1
/books/4139000033?_rdt_sid=bestseller&_rdt_idx=2
/books/1890000285?_rdt_sid=bestseller&_rdt_idx=3
/books/4575000003?_rdt_sid=bestseller&_rdt_idx=4
/books/1648000475?_rdt_sid=bestseller&_rdt_idx=5
/books/4803001974?_rdt_sid=bestseller&_rdt_idx=6
/books/2177000101?_rdt_sid=bestseller&_rdt_idx=7
/books/1176000018?_rdt_sid=bestseller&_rdt_idx=8
/books/1546000953?_rdt_sid=bestseller&_rdt_idx=9
/books/4578000004?_rdt_sid=bestseller&_rdt_idx=10
/books/371001540?_rdt_sid=bestseller&_rdt_idx=11
/books/4926000001?_rdt_sid=bestseller&_rdt_idx=12
/books/4912000001?_rdt_sid=bestseller&_rdt_idx=13
/books/1508006000?_rdt_sid=bestseller&_rdt_idx=14
/books/745000210?_rdt_sid=bestseller&_rdt_idx=15
/books/4239000039?_rdt_sid=bestseller&_rdt_idx=16
/books/443001038?_rdt_sid=bestseller&_rdt_idx=17
/books/4647000003?_rdt_sid=bestseller&_rdt_idx=18
/books/4909000001?_rdt_sid=bestseller&_rdt_idx=19
/books/6060023

#### 선생님 풀이

In [35]:
hyperlink_lists = soup.select(".title_link") 
print(hyperlink_lists[:5])

[<a class="title_link" data-track-params="" data-track-type="" href="/books/1468000044?_rdt_sid=bestseller_star&amp;_rdt_idx=0">
<span class="title_text js_highlight_helper hidden">          
                        원씽
            </span>
</a>, <a class="title_link" data-track-params="" data-track-type="" href="/books/606002388?_rdt_sid=bestseller&amp;_rdt_idx=0">
<span class="title_text js_highlight_helper hidden">          
                        역행자
            </span>
</a>, <a class="title_link" data-track-params="" data-track-type="" href="/books/4271000014?_rdt_sid=bestseller&amp;_rdt_idx=1">
<span class="title_text js_highlight_helper hidden">          
                        작별인사
            </span>
</a>, <a class="title_link" data-track-params="" data-track-type="" href="/books/4139000033?_rdt_sid=bestseller&amp;_rdt_idx=2">
<span class="title_text js_highlight_helper hidden">          
                        인플레이션에서 살아남기
            </span>
</a>, <a class="title_link" data

In [14]:
# star book 제거
hyperlink_lists = hyperlink_lists[1:]

In [15]:
hyperlink = []
ridiURL = "https://ridibooks.com"
for hyperlink_list in hyperlink_lists:
    hyperlink.append(ridiURL + hyperlink_list['href'])

In [16]:
hyperlink

['https://ridibooks.com/books/606002388?_rdt_sid=bestseller&_rdt_idx=0',
 'https://ridibooks.com/books/4271000014?_rdt_sid=bestseller&_rdt_idx=1',
 'https://ridibooks.com/books/4139000033?_rdt_sid=bestseller&_rdt_idx=2',
 'https://ridibooks.com/books/1890000285?_rdt_sid=bestseller&_rdt_idx=3',
 'https://ridibooks.com/books/4575000003?_rdt_sid=bestseller&_rdt_idx=4',
 'https://ridibooks.com/books/1648000475?_rdt_sid=bestseller&_rdt_idx=5',
 'https://ridibooks.com/books/4803001974?_rdt_sid=bestseller&_rdt_idx=6',
 'https://ridibooks.com/books/2177000101?_rdt_sid=bestseller&_rdt_idx=7',
 'https://ridibooks.com/books/1176000018?_rdt_sid=bestseller&_rdt_idx=8',
 'https://ridibooks.com/books/1546000953?_rdt_sid=bestseller&_rdt_idx=9',
 'https://ridibooks.com/books/4578000004?_rdt_sid=bestseller&_rdt_idx=10',
 'https://ridibooks.com/books/371001540?_rdt_sid=bestseller&_rdt_idx=11',
 'https://ridibooks.com/books/4926000001?_rdt_sid=bestseller&_rdt_idx=12',
 'https://ridibooks.com/books/4912000

In [36]:
# 1번 책으로 hyper link test

url = hyperlink[0]
res = req.urlopen(url)
soup = BeautifulSoup(res, "html.parser")

book_contents = soup.select_one(".folded")

### 작품소개 불러오기

In [18]:
# #introduce_book > p
# 클래스는 띄어쓰기로 구분되어 있네!?
# 
# p class 는 하나짜리이고 p안에 br 이 들어가있네 이런거를 처리하는게 .text 로 해결해야함. 
# 하지만 string이 퍼포먼스가 빠름

book_link = soup.select_one(".folded")

In [19]:
# text는 하나로 가져온것만 바꿀수 있다.
book_link.text

'<역행자> 오타쿠 흙수저에서 월 1억 자동수익을 실현한 무자본 연쇄창업마,\r\n라이프해커 자청의 인생 역주행 공식 대공개!\r\n“절대 읽지 마라!\r\n죽을 때까지 순리자로 살고 싶다면!”\n\r\n운명과 본능의 지배에서 벗어나\r\n경제적 자유와 행복한 인생을 쟁취하는 라이프해킹의 비밀\n\r\n95퍼센트의 인간은 타고난 유전자와 본성의 꼭두각시로 살아간다. 그래서 평생 평범함을 벗어나지 못하고 불행하게 산다. 본성에 따라 결정된 인생을 사는 자, 이들은 ‘순리자’다. 그러나 5퍼센트의 인간은 다르다. 그들은 타고난 유전자와 본성을 역행해 경제적 자유와 행복을 쟁취한다. 이들이 바로 ‘역행자’다.\r\n어느 날 혜성처럼 나타나 자수성가의 아이콘이 된 청년, 자기계발 유튜버 자청이 깨달은 인생의 치트기를 탈탈 털어 넣은 그의 첫 책 《역행자》가 5월 30일, 드디어 출간된다. 수많은 ‘자청 챌린저’들이 기다려온 책이자, 독자들의 인생을 송두리째 바꿔줄 문제작이다. ‘연봉 10억 무자본 창업가’로 유명해진 그는 2020년, 단 20개의 영상으로 10만 구독자를 넘어서며 화제를 모았으나 곧장 미련 없이 유튜브를 그만뒀다. 이후 본업인 온라인 마케팅 비즈니스를 비롯해 다방면으로 사업을 확장하고 있다.\r\n이 책에는 가난한 인생에서 벗어나 경제적 자유와 행복을 얻은 저자 자청이 깨달은 인생 레벨업 치트키가 빼곡히 담겨 있다. 10대 때의 그는 외모, 돈, 공부, 그 어떤 것에서도 최하위였다. 그러던 스무 살 무렵, ‘인생에도 게임처럼 공략집이 있다’는 사실을 깨닫고 삶이 180도 바뀌기 시작했다. 200여 권의 책을 독파하며 얻은 인생의 치트키들을 활용해 창업에 성공했다.\n\r\n‘부자는 다른 세상 사람들이야. 우리 집 꼴을 봐’, ‘인서울 대학에 가는 건 천재들이나 하는 거야’, ‘내가 월 300 이상 버는 날은 영원히 없겠지’… 인생의 밑바닥에서 힘든 친구들은 비슷한 심정일 거라 생각한다. 하지만 지금 내 정체성은 완전히 다르다. 나는 평생에 걸쳐 정체성

In [37]:
# Test후 적용
bookContents = []

for url in hyperlink:
    res = req.urlopen(url)
    soup = BeautifulSoup(res, "html.parser")
    book_contents = soup.select_one(".folded")
    bookContents.append(book_contents.text)

In [38]:
bookContents[:5]

['<역행자> 오타쿠 흙수저에서 월 1억 자동수익을 실현한 무자본 연쇄창업마,\r\n라이프해커 자청의 인생 역주행 공식 대공개!\r\n“절대 읽지 마라!\r\n죽을 때까지 순리자로 살고 싶다면!”\n\r\n운명과 본능의 지배에서 벗어나\r\n경제적 자유와 행복한 인생을 쟁취하는 라이프해킹의 비밀\n\r\n95퍼센트의 인간은 타고난 유전자와 본성의 꼭두각시로 살아간다. 그래서 평생 평범함을 벗어나지 못하고 불행하게 산다. 본성에 따라 결정된 인생을 사는 자, 이들은 ‘순리자’다. 그러나 5퍼센트의 인간은 다르다. 그들은 타고난 유전자와 본성을 역행해 경제적 자유와 행복을 쟁취한다. 이들이 바로 ‘역행자’다.\r\n어느 날 혜성처럼 나타나 자수성가의 아이콘이 된 청년, 자기계발 유튜버 자청이 깨달은 인생의 치트기를 탈탈 털어 넣은 그의 첫 책 《역행자》가 5월 30일, 드디어 출간된다. 수많은 ‘자청 챌린저’들이 기다려온 책이자, 독자들의 인생을 송두리째 바꿔줄 문제작이다. ‘연봉 10억 무자본 창업가’로 유명해진 그는 2020년, 단 20개의 영상으로 10만 구독자를 넘어서며 화제를 모았으나 곧장 미련 없이 유튜브를 그만뒀다. 이후 본업인 온라인 마케팅 비즈니스를 비롯해 다방면으로 사업을 확장하고 있다.\r\n이 책에는 가난한 인생에서 벗어나 경제적 자유와 행복을 얻은 저자 자청이 깨달은 인생 레벨업 치트키가 빼곡히 담겨 있다. 10대 때의 그는 외모, 돈, 공부, 그 어떤 것에서도 최하위였다. 그러던 스무 살 무렵, ‘인생에도 게임처럼 공략집이 있다’는 사실을 깨닫고 삶이 180도 바뀌기 시작했다. 200여 권의 책을 독파하며 얻은 인생의 치트키들을 활용해 창업에 성공했다.\n\r\n‘부자는 다른 세상 사람들이야. 우리 집 꼴을 봐’, ‘인서울 대학에 가는 건 천재들이나 하는 거야’, ‘내가 월 300 이상 버는 날은 영원히 없겠지’… 인생의 밑바닥에서 힘든 친구들은 비슷한 심정일 거라 생각한다. 하지만 지금 내 정체성은 완전히 다르다. 나는 평생에 걸쳐 정체

In [ ]:
ridibooks['Contents'] = pd.Series(bookContents)
ridibooks

,Rank,bookTitle,Contents
0,1,역행자,"<역행자> 오타쿠 흙수저에서 월 1억 자동수익을 실현한 무자본 연쇄창업마,\r\n라..."
1,2,작별인사,"<작별인사> 누구도 도와줄 수 없는 상황, 혼자 헤쳐나가야 한다\r\n지켜야 할 약..."
2,3,인플레이션에서 살아남기,"<인플레이션에서 살아남기> \n\n40년 만에 찾아온 인플레의 역습, \r\n당신의..."
3,4,변화하는 세계 질서,<변화하는 세계 질서> \n\n《원칙 Principles: Life & Work》으...
4,5,피스타운,"<피스타운> -“여긴 너무 비현실적이야, 그렇지 않아?”\r\n-뭐든지 이룰 수 있..."
